In [12]:
import os

In [16]:
os.chdir("/Users/pandhari/Desktop/MLOPs/End-To_End_Projects/Text_Summariser")

In [17]:
%pwd

'/Users/pandhari/Desktop/MLOPs/End-To_End_Projects/Text_Summariser'

In [18]:
from dataclasses import dataclass
from pathlib import Path

In [19]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs : int
    warmmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [20]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml , create_directories

In [21]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
 

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir , 
            data_path=config.data_path , 
            model_ckpt=config.model_ckpt , 
            num_train_epochs=params.num_train_epochs , 
            warmmup_steps=params.warmmup_steps , 
            per_device_train_batch_size=params.per_device_train_batch_size , 
            weight_decay=params.weight_decay , 
            logging_steps=params.logging_steps , 
            evaluation_strategy=params.evaluation_strategy , 
            eval_steps=params.eval_steps , 
            save_steps=params.save_steps , 
            gradient_accumulation_steps=params.gradient_accumulation_steps  
        )

        return model_trainer_config

    

In [22]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk

In [26]:
class ModelTrainer:
    def __init__(self , config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)


        # Loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,  # Minimized batch size
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=4,
        )


        trainer = Trainer ( model=model_pegasus, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=dataset_samsum_pt["test"],
                    eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir , "pegasus-samsum-model"))


        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir , "tokenizer"))




In [27]:
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate


In [28]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2024-12-12 17:48:25,139: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-12 17:48:25,147: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-12 17:48:25,150: INFO: common: created directory at : artifacts]
[2024-12-12 17:48:25,151: INFO: common: created directory at : artifacts/model_trainer]


python(8507) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/42/94lj0cp519ncj7_m79j3_h3m0000gn/T/ipykernel_6382/3938076124.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer ( model=model_pegasus, args=trainer_args,


ValueError: fp16 mixed precision requires a GPU (not 'mps').